# *[Using AutoML as a start point]*

**Author:** [Marco Bertani-Økland](https://github.com/mbertani)

**Achievement:** Illustrate the use of AutoML as a starting point to explore different algorithms.

## Introduction

This notebook is based on [https://supervised.mljar.com/](https://supervised.mljar.com/).

Run the notebook and check the results produced under the folder `results_diabetes`. 

Requirements:

1. You must run `make venv` to verify that all packages are installed.
2. You must have downloaded the [diabetes dataset](https://www.kaggle.com/datasets/alexteboul/diabetes-health-indicators-dataset) into the folder `NBD_22_workshop`.

# Reproducibility and code formatting

In [8]:
# To watermark the environment
%load_ext watermark

# For automatic code formatting in jupyter lab.
%load_ext lab_black

# For automatic code formatting in jupyter notebook
%load_ext nb_black

# For better logging
%load_ext rich

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black
The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black
The rich extension is already loaded. To reload it, use:
  %reload_ext rich


# Analysis

In [9]:
# Imports
# -------

# System
import sys

# Logging
import logging

# Rich logging in jupyter
from rich.logging import RichHandler

FORMAT = "%(message)s"
logging.basicConfig(
    level="INFO", format=FORMAT, datefmt="[%X]", handlers=[RichHandler()]
)

log = logging.getLogger("rich")

# Nice logging example:
# log.error("[bold red blink]Server is shutting down![/]", extra={"markup": True})


# Other packages
import pandas as pd
from sklearn.model_selection import train_test_split
from supervised.automl import AutoML

In [10]:
datapath = "../NBD_22_workshop/diabetes_binary_5050split_health_indicators_BRFSS2015.csv"
df = pd.read_csv(datapath)

In [11]:
target_column = "Diabetes_binary"
train_columns = list(df.columns)
train_columns.remove(target_column)

In [12]:
target_column = "Diabetes_binary"
train_columns = list(df.columns)
train_columns.remove(target_column)
X_train, X_test, y_train, y_test = train_test_split(
    df[train_columns], df[target_column], test_size=0.25, random_state=42
)

automl = AutoML(results_path="results_diabetes")
automl.fit(X_train, y_train)

predictions = automl.predict(X_test)

Linear algorithm was disabled.
This model has already been fitted. You can use predict methods or select a new 'results_path' for a new 'fit()'.


X has feature names, but StandardScaler was fitted without feature names


# Watermark

This should be the last section of your notebook, since it watermarks all your environment.

When commiting this notebook, remember to restart the kernel, rerun the notebook and run this cell last, to watermark the environment.

In [13]:
%watermark -gb -iv -m -v

Python implementation: CPython
Python version       : 3.8.13
IPython version      : 8.4.0

Compiler    : GCC 10.3.0
OS          : Linux
Release     : 5.15.0-1017-aws
Machine     : x86_64
Processor   : x86_64
CPU cores   : 4
Architecture: 64bit

Git hash: 31021eb0e442bc8b52c35efe4f0ac391b65a60f0

Git branch: add-automl-example

sys    : 3.8.13 | packaged by conda-forge | (default, Mar 25 2022, 06:04:10) 
[GCC 10.3.0]
logging: 0.5.1.2
pandas : 1.4.3

